# Seq2Seq 모델을 사용한 챗봇 구현 튜토리얼

코드&설명 제공 (Thanks to)
- original code by 이영준 (2020 KEPCO 문제해결형 인공지능기술개발교육 TA)
- 토크나이저 변경 by 조상현 (부산대학교 AI대학원 자연어처리 TA)


본 실습에서는 sequence-to-sequence (seq2seq) 모델을 이용하여 생성 챗봇을 구현한다.

학습데이터로 일상대화 데이터 쌍 (약 12,000건)을 사용한다.

**생성 챗봇 대화 예시**: 
<pre>
<code>
문장을 입력하세요: 안녕
Bot: 안녕하시어요.
문장을 입력하세요: 갈까 말까?
Bot: 가시어요.
문장을 입력하세요: 12시 땡!
Bot: 하루가 또 가네요.
</code>
</pre>

## 원본 코드, 데이터 출처
[chatbot 튜토리얼]: https://pytorch.org/tutorials/beginner/chatbot_tutorial.html#
[한국어 대화 데이터]: https://github.com/songys/Chatbot_data

- 본 실습 코드는 PyTorch 에서 제공하는 [chatbot 튜토리얼]을 참고하였습니다.
- seq2seq 모델의 학습을 위해 [한국어 대화 데이터]를 사용하였습니다.

## Step 0: Connect to Google drive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Step 1: Import module

re : 정규표현식 사용을 위한 내장 모듈

collections : 파이썬의 자료형(list, tuple, dict)들에게 확장된 기능을 제공하는 내장 모듈. vocab 구축 시 사용 

os : 기본 내장 모듈로서 경로생성 함수 제공

In [ ]:
# 라이브러리 호출
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import re
import random
import os
import csv
import pickle as pc
import collections
import numpy as np

# reproducibility (재실행시마다 동일한 결과를 얻기 위해서)
SEED = 470
random.seed(SEED)                 # python random 라이브러리의 random seed를 고정
np.random.seed(SEED)              # numpy에서 난수 생성시 random seed를 고정
torch.manual_seed(SEED)           # CPU 연산시 pytorch에서 random seed를 고정
torch.cuda.manual_seed(SEED)      # GPU 연산시 pytorch에서 random seed를 고정
torch.cuda.manual_seed_all(SEED)  # 멀티 GPU 연산 시 random seed를 고정

# torch 버전 확인
print("Pytorch Version: ", torch.__version__)

# GPU 사용 가능 여부 확인
if torch.cuda.is_available():
    # PyTorch GPU 사용 설정
    device = torch.device("cuda")
    print("There are %d GPU(s) available." % torch.cuda.device_count())
    print("We will use the GPU:", torch.cuda.get_device_name(0))
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")

Pytorch Version:  1.12.1+cu113
There are 1 GPU(s) available.
We will use the GPU: A100-SXM4-40GB


## Step 2: Configure the experiments

- 모델의 실험(학습 및 평가)을 필요한 파라미터 및 인자 설정
    - Hyperparameter: hidden size, vocabulary size, max length, dropout rate 등
    - Argument: file directory 등

In [ ]:
# 데이터, 모델 위치 (직접 설정이 필요함)
data_dir = '/content/gdrive/My Drive/Colab Notebooks/aivle/data/songys/ChatbotData.csv'
dirpath = '/content/gdrive/My Drive/Colab Notebooks/aivle/model/'

if not os.path.exists(dirpath):   
    os.makedirs(dirpath)    # dirpath가 없으면 해당 디렉토리를 생성해줌

# word2idx : binary file (단어 -> index로 매핑)    
WORD_DICT_DIR = '/content/gdrive/My Drive/Colab Notebooks/aivle/data/songys/word2idx'

THRESHOLD = 40000 #dictionary에서 토큰 갯수의 한계값 설정
MAX_LEN = 25  #문장의 최대 길이

attn_model = 'dot'  # 어텐션 기법 : dot 프로덕트

#실행이 오래걸리지 않도록 인코더와 디코더의 hidden_size, layer_number를 작게 설정
enc_hidden_size = 200
dec_hidden_size = 400  # 인코더가 양방향이므로
encoder_n_layers = 1  
decoder_n_layers = 1

dropout = 0.1 # 랜덤하게 정한 Weight의 10퍼센트는 사용하지 않음 -> 오버피팅 방지
batch_size = 32 #연산 한 번에 들어가는 학습 데이터의 갯수 32개
max_epochs = 10 #전체 학습데이터 셋이 신경망을 통과한 횟수

learning_rate = 0.001 #학습률 : 옵티마이저에서 파라미터값을 변경하는 비율



## Step 3: Data preparation

모델의 학습을 위해 데이터를 준비하는 과정이고 다음과 같다.
    
   - Load data
   - Tokenization
   - Build vocab

### Step 3-1: Load data

- **"ChatbotData.csv"** 파일 load
- **`[utterance, response]`** pair 의 형태로 재구성

In [ ]:
# pair data load
pair_data = list() # 비어 있는 list 생성

f = open(data_dir, 'r', encoding='utf-8')
reader = csv.reader(f)      # reader에서 콤마(,)를 구분자로 질문, 대답 부분을 읽어옴
for idx, line in enumerate(reader):
    if idx == 0:    # 첫번째 라인은 skip (파일 포맷에 대한 코멘트 라인)
        continue
        
    pair_data.append([line[0], line[1]])
f.close()

# pair data 확인
print(pair_data[:3])  # pair_data는 리스트의 리스트 타입

[['12시 땡!', '하루가 또 가네요.'], ['1지망 학교 떨어졌어', '위로해 드립니다.'], ['3박4일 놀러가고 싶다', '여행은 언제나 좋죠.']]


### Step 3-2: Tokenization

한국어 Word Piece 방식 토크나이저를 사용합니다.

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 5.0 MB/s 
     |████████████████████████████████| 163 kB 82.0 MB/s 
     |████████████████████████████████| 6.6 MB 71.8 MB/s 


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("monologg/kobigbird-bert-base")

Downloading:   0%|          | 0.00/373 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/241k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/492k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/169 [00:00<?, ?B/s]

In [ ]:
# 형태소 분석된 [질문, 응답] 데이터셋 구축
total_data = list()   # 빈 리스트를 생성
for each in pair_data:
    utter = tokenizer.tokenize(each[0])
    resp = tokenizer.tokenize(each[1])
    total_data.append([utter, resp])
    
# 데이터 사이즈 및 실제 결과 확인
print("Total size of data is", len(total_data))
print("\nExample:")
print(total_data[:1])

Total size of data is 11823

Example:
[[['12', '##시', '땡', '!'], ['하루', '##가', '또', '가네', '##요', '.']]]


### Step 3-3: Data split & shuffling 

[링크]: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

학습 및 평가를 위해 원본 데이터에서 **90%** 는 학습 데이터로 사용하고, **10%** 는 평가 데이터로 사용합니다. 이를 위해 `sklearn` 라이브러리에 `train_test_split` 함수를 사용하고, 함수의 argument 정보들은 해당 [링크]에서 확인할 수 있습니다.


In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(total_data, test_size=0.1, random_state=42, shuffle=True)

# 학습 및 평가 데이터 크기 확인
print("Train/Test size is {}/{}".format(len(train), len(test)))
print("\nExample:")
print(train[:2])

Train/Test size is 10640/1183

Example:
[[['나', '##한', '##테', '질리', '##면', '어쩌', '##지', '걱정', '##돼'], ['당신', '##의', '겉모습', '##이', '아닌', '진정한', '내면', '##의', '모습', '##을', '보여', '##주', '##세요', '.']], [['새로운', '일', '벌려', '##도', '될까'], ['도전', '##해', '봐도', '좋', '##을', '거', '같', '##아', '##요', '.']]]


### Step 3-4: Create Word Dictionary

실제 자연어로 이루어진 단어들을 기계가 이해할 수 있는 index 값으로 맵핑해주는 dictionary를 구축합니다. 추가로, 응답을 생성하는 단계를 위해 index가 단어로 맵핑되는 dictionary도 구축합니다.

- `build_dict`: 학습 데이터의 토큰들을 빈도수 단위로 내림차순 정렬하여, 위에서부터 **threshold** 기준으로 dictionary 갯수/사이즈를 지정한다.
    - special tokens
        - **pad**: GPU를 이용하여 모델을 학습시키기 위해서는 batch 내에 있는 모든 문장들이 동일한 길이를 가져야한다. 이를 위해, maximum 길이를 지정하고 남은 부분은 padding token 을 채워줍니다. 주의할 점은, 학습시 loss 를 구할 때 padding 에 해당되는 부분은 반영시키지 않아야 합니다.
        - **unk**: word dictionary 에 없는 단어(token)가 등장하면 unknown token 을 채워줍니다.
        - **sos**: 디코더에서 문장의 시작을 알리는 토큰입니다.
        - **eos**: 디코더에서 문장의 끝을 알리는 토큰입니다.

In [ ]:
EOS_token = 3
SOS_token = 2
UNK_token = 1
PAD_token = 0

def build_dict(data, threshold=40000):
    
    if not os.path.exists(WORD_DICT_DIR) or True:
        """
        Build word dictionary
        """
        
        vocab = list()
        for doc in data:
            for word in doc[0]:
                vocab.append(word)
            for word in doc[1]:
                vocab.append(word)
        # 단어의 빈도수 순서로 vocab을 sorting한다. vocab의 최대 크기는 threshold
        counter = collections.Counter(vocab).most_common(threshold)
        
        word2idx = dict()
        word2idx['<pad>'] = PAD_token #0
        word2idx['<unk>'] = UNK_token #1
        word2idx['<sos>'] = SOS_token #2
        word2idx['<eos>'] = EOS_token #3
        
        # 고빈도 단어부터 ID를 부여함
        for word, _ in counter:
            word2idx[word] = len(word2idx)
        
        with open(WORD_DICT_DIR, 'wb') as f:
            pc.dump(word2idx, f)
    else:
        """
        Load word dictionary which was built before
        """
        with open(WORD_DICT_DIR, 'rb') as f:
            word2idx = pc.load(f)
    
    print("Load word dictionary")
    return word2idx     # (단어 : index) 포맷의 사전을 반환

In [ ]:
# build word2idx, idx2word (생성시에 사용)
# train : 학습데이터셋
# word -> index 매핑 사전
word2idx = build_dict(train, THRESHOLD)
# index -> word 매핑 사전
idx2word = {idx: word for word, idx in word2idx.items()}

# 사전 사이즈
vocab_size = len(word2idx)
print("The size of word2idx is {}".format(len(word2idx)))

Load word dictionary
The size of word2idx is 6093


In [ ]:
print(idx2word[0])
print(idx2word[100])
print(idx2word[6092])


<pad>
저
비례


## Step 4: Prepare data for Model

모델의 학습을 위해 학습 데이터를 word dictionary 를 이용하여 index 로 변환합니다. 그리고 학습 데이터를 mini-batch 단위로 준비합니다.

- `batch_iter`: 학습 과정에서 iteration 돌 때마다, 배치 단위의 데이터를 불러오는 과정을 위한 함수
- `batch_dataset`: mini-batch 단위 학습 데이터를 만들어주는 함수
    - 모델 학습의 efficiency 를 위해 **MAX_LEN** 만큼 데이터 자르기
    - 처음 보는 단어는 **unk** 토큰으로 채우기
    - utterance 문장의 뒤에 **eos** 토큰 더하기
    - response 문장의 앞/뒤에 **sos/eos** 토큰 더하기
    - **MAX_LEN** 까지 남은 부분은 **pad** 토큰 채우기

In [ ]:
def batch_iter(data, batch_size): 
    num_batches_per_epoch = int(len(data) / batch_size)
    # data : 현재 스트링 타입 -> array 타입으로 저장 (이후에 텐서로 변환된다)
    data = np.array(data)
    
    # num_batches_per_epoch = 332 
    for batch_idx in range(num_batches_per_epoch):
        # start_idx = 0, 32, 64, ...
        start_idx = batch_idx * batch_size
        # ebd_idx = 32, 64, 96, ...
        end_idx = min((batch_idx + 1) * batch_size, len(data))
        enc_data = list()   # 인코더 데이터
        dec_data = list()   # 디코더 데이터
        
        for each in data[start_idx:end_idx]:
            enc_data.append(each[0])  # utterence
            dec_data.append(each[1])  # response

        yield enc_data, dec_data  # 인코더 데이터, 디코더 데이터 반환하는 것을 반복
        
# 인코더 데이터, 디코더 데이터 -> index로 바꾼 후 array 타입으로 반환하는 함수
def batch_dataset(batch_x, batch_y, word2idx):
    # batch input & target
    # map(함수, 리스트) : 리스트로부터 원소를 하나씩 꺼내서 함수를 적용시킨 다음, 그 결과를 새로운 리스트에 넣는다
    batch_x = list(map(lambda x: x[:MAX_LEN], batch_x)) # utterence 문장을 하나씩 꺼내서 MAX_LEN 사이즈로 자른다
    batch_y = list(map(lambda x: x[:MAX_LEN], batch_y)) # response 문장을 하나씩 꺼내서 MAX_LEN 사이즈로 자른다

    # utterence 문장, response 문장 -> index로 변환
    # word2idx 딕셔너리에서 찾는 단어가 없을 땐 디폴트값인 '<unk>'의 index를 반환
    batch_x = list(map(lambda x: [word2idx.get(each, word2idx['<unk>']) for each in x], batch_x))
    batch_y = list(map(lambda x: [word2idx.get(each, word2idx['<unk>']) for each in x], batch_y))
                        
    batch_enc_input = list(map(lambda x: list(x) + [word2idx['<eos>']], batch_x))            
    batch_dec_target = list(map(lambda x: [word2idx['<sos>']] + list(x) + [word2idx['<eos>']], batch_y))

    # MAX_LEN보다 짧은 문장이라면 <pad> 채워넣기        
    batch_enc_input = list(map(lambda x: list(x) + (MAX_LEN+1 - len(x)) * [word2idx['<pad>']], batch_enc_input))         
    batch_dec_target = list(map(lambda x: list(x) + (MAX_LEN+2 - len(x)) * [word2idx['<pad>']], batch_dec_target))
    
    batch_enc_input = np.array(batch_enc_input) # integer 리스트 --> array로 변환
    batch_dec_target = np.array(batch_dec_target)
    
    return batch_enc_input, batch_dec_target # integer 값이 저장된 array 반환


## Step 5: Building seq2seq model with attention mechanism


In [ ]:
# Encoder RNN 클래스 정의, nn.Module을 상속
class EncoderRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__() # 부모 클래스 nn.Module을 초기화
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.embedding = nn.Embedding(vocab_size, hidden_size) # vocab size x 히든사이즈 matrix, 랜덤값으로 초기화
        
        # Initialize LSTM with bidirectional
        # the input_size and hidden_size params are both set to 'hidden_size'
        # because our input size is a word embedding with number of features == hidden_size
        # nn.LSTM (input_size, hidden_size, n_layers, bidirectional, droupout, batch_first)
        # batch_first가 True -> 텐서 1번째 차원이 배치 사이즈인 것을 알림
        # input tensors, output tensors -> (batch_size, seq_length, embed_size)
        # Note that this does not apply to hidden or cell states. 
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers, bidirectional=True, dropout=dropout, batch_first=True)

    def forward(self, enc_input):
        # Convert word indexed to embeddings (mapping discrete tokens to continuous space)
        # embedded shape == (batch_size, enc_max_len, embed_size)
        embedded = self.embedding(enc_input)
       
        # Forward pass through RNN module
        # if bidirectional, outputs shape == (batch_size, seq_length, hidden_size*2)
        # if not bidirectional, outputs shape == (batch_size, seq_length, hidden_size)
        # hidden shape == (num_directions * num_layers, batch_size, hidden_size)
        outputs, (hidden, cell) = self.lstm(embedded)

        
        # output of shape (batch_size, seq_length, num_directions * hidden_size)
        # h of shape (num_layers * num_directions, batch, hidden_size)
        # c of shape (num_layers * num_directions, batch, hidden_size)
        return outputs, (hidden, cell)


In [ ]:
# Luong attention layer
class Attention(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attention, self).__init__()
        self.method = method
        # method: indicator that determines the score function 
        
        self.hidden_size = hidden_size          

    def dot_score(self, hidden, encoder_output):        
        # attention dot score function (Luong)
        # attention score shape == (batch_size, dec_max_len)
        # sum: returns the sum of each row of the input tensor in the given dimension dim
        return torch.sum(hidden * encoder_output, dim=2) # 텐서의 dimension 2가 squeeze

    def forward(self, hidden, encoder_outputs):
        attn_weights = self.dot_score(hidden, encoder_outputs)
        
        # Transpose max_length and batch_size dimensions
        attention = attn_weights

        # Normalize attention to weights in range 0 to 1
        return F.softmax(attention, dim=1).unsqueeze(1) # attention을 3D shape로 변환 (resize to 1 x 1 x seq_length)

In [ ]:
# Decoder Model
class AttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, hidden_size, vocab_size, n_layers=1, dropout=0.1):
        super(AttnDecoderRNN, self).__init__()
        
        self.attn_model = attn_model # 'dot' 프로덕트
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.n_layers = n_layers
        self.dropout = dropout
        
        # Define layers
        self.embedding = nn.Embedding(vocab_size, hidden_size)
             
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size, bidirectional=False, num_layers=1, batch_first=True)
        self.concat = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.vocab_size)

        self.attn = Attention(self.attn_model, self.hidden_size)
    
    def forward(self, dec_input, hidden, cell, encoder_outputs):
        # Note: we run this one step (word) at a time
        
        # Get embedding of current input word
        # embedded shape == (batch_size, 1, embed_size)
        embedded = self.embedding(dec_input)
    
        # Forward through unidirectional LSTM
        # output shape == (batch_size, 1, hidden_size)
        # hidden shape == (num_directions * num_layers, batch_size, hidden_size)        
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))

        # Calculate attention weights from the current LSTM output
        # 현 시점의 디코더 hidden(Query)과 인코더의 모든 hidden(Key)을 dot product -> softmax -> attention weights
        # attention weights shape == (batch_size, 1, dec_max_len)
        attn_weights = self.attn(output, encoder_outputs) 

        # Multiply attention weights to encoder outputs to get new "weighted sum(가중합)" context vector
        # context vector shape == (batch_size, 1, hidden_size)
        # torch.bmm 함수는 배치 행렬 곱(Batch Matrix Multiplication, BMM)을 수행하는 함수로서
        # 뒤의 두 개 차원에 대해 행렬 곱을 수행함 
        # attn_weights X 인코더 모든 hidden states -> context 
        context = attn_weights.bmm(encoder_outputs)  
        
        # Concatenate weighted context vector and LSTM output using Luong
        # output shape == (batch_size, 1, hidden_size)
        # context shape == (batch_size, 1, hidden_size) 
        # concat_input shape == (batch_size, hidden_size * 2)
        # concat_output shape == (batch_size, hidden_size)
        output = output.squeeze(1)    # dimension 1의 차원을 제거
        context = context.squeeze(1)  # dimension 1의 차원을 제거
        concat_input = torch.cat((output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        
        # Predict next word using Luong
        # output shape == (batch_size, vocab_size)
        output = self.out(concat_output)
        output = F.log_softmax(output, dim=1) # dimension 1의 값을 softmax (dimension 0는 batch_size)
        
        # Return output and final hidden state
        return output, hidden, cell

## Step 6: Define the optimizer and the loss function

- optimizer: Adam 사용
- loss function: negative log likelihood 사용

In [ ]:
# Initialize encoder & decoder models
# model.cuda() : model의 모든 parameter를 GPU에 loading
encoder = EncoderRNN(vocab_size, enc_hidden_size, encoder_n_layers, dropout).cuda() 
decoder = AttnDecoderRNN(attn_model, dec_hidden_size, vocab_size, decoder_n_layers, dropout).cuda()

print("Models are built and ready to go!")

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Models are built and ready to go!


In [ ]:
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)

criterion= nn.NLLLoss()

## Step 7: Training with evaluation

- evaluation: training with evaluation function

In [ ]:
def evaluate(encoder, decoder, val_batches, device, word2idx):
    #evaluating the validation loss
    total_loss = 0.
    
    for batch_idx, (batch_x, batch_y) in enumerate(val_batches):
        if batch_idx == 1:
            break
        # 인코더 데이터, 디코더 데이터 -> index로 바꾼 후 array 타입으로 반환하는 함수    
        batch_enc_input, batch_dec_target = batch_dataset(batch_x, batch_y, word2idx)
        
        # long 타입의 텐서를 생성해서 GPU에 로딩
        # batch_enc_input : utterence
        # batch_dec_target : response
        batch_enc_input = torch.tensor(batch_enc_input, dtype = torch.long, device='cuda')
        batch_dec_target = torch.tensor(batch_dec_target, dtype = torch.long, device='cuda')
        
        loss = 0.
        
        with torch.no_grad():
            # Forward pass through encoder
            encoder_outputs, (encoder_hidden, encoder_cell) = encoder(batch_enc_input)

            # torch.cat(tensors, dim=0) : 
            # Concatenates the given sequence of tensors in the given dimension. 
            # dim: the dimension over which the tensors are concatenated          
            eh = torch.cat((encoder_hidden[0], encoder_hidden[1]), dim=1).unsqueeze(0)
            ec = torch.cat((encoder_cell[0], encoder_cell[1]), dim=1).unsqueeze(0)
            
            # Create initial decoder input (start with <sos> tokens for each sentence)
            dec_input = torch.tensor([word2idx['<sos>']] * batch_size, dtype=torch.long, device=device)
            dec_input = dec_input.unsqueeze(1)                  # dec_input shape : (? , ?)

            # Set initial decoder hidden state to the encoder's final hidden state
            decoder_hidden = encoder_hidden[:decoder.n_layers]  # encoder_hidden[0]

            # Tensor.size(dim=None) : If dim is specified, returns the size of that dimension.
            # 즉, response의 max_len을 반환            
            for t in range(1, batch_dec_target.size(1)):
                decoder_output, eh, ec = decoder(dec_input, eh, ec, encoder_outputs)

                # Calculate and accumulate loss
                loss += criterion(decoder_output, batch_dec_target[:, t])

                # Teacher forcing: next input is current target
                dec_input = batch_dec_target[:, t].unsqueeze(1)
 
            # print (batch_dec_target.size(1))
            # item() method extracts the loss’s value as a Python float    
            batch_loss = loss.item() / int(batch_dec_target.size(1))
            total_loss += batch_loss
        
    # total_loss를 batch 갯수로 나누어 loss 평균값을 반환
    return total_loss / (batch_idx + 1)    

In [ ]:
%%time

teacher_forcing_ratio = 1.0
LOG_INTERVAL = 100

num_batches_per_epoch = int(len(train) / batch_size)
print("[num_batches_per_epoch] {}".format(num_batches_per_epoch))

# Training Epoch: 10
for epoch in range(max_epochs):

    total_loss = 0.
    total_ppl = 0.
    print_loss = 0.
    
    # batch_iter : 학습 과정에서 iteration 돌 때마다, 배치 단위의 데이터를 불러오는 함수
    train_batches = batch_iter(train, batch_size)
    
    # Set models as training mode
    encoder.train()
    decoder.train()
    
    # Training loop
    for batch_idx, (batch_x, batch_y) in enumerate(train_batches):
      
        batch_enc_input, batch_dec_target = batch_dataset(batch_x, batch_y, word2idx)
        
        # Initialize variables
        loss = 0.
        
        # zero gradients
        # Iteration이 한번 끝나면 gradients를 항상 0으로 reset해야 함 
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        # batch_enc_input shape : (batch_size, seq_length)
        # batch_dec_target shape : (batch_size, seq_length)
        batch_enc_input = torch.tensor(batch_enc_input, dtype = torch.long, device='cuda')
        batch_dec_target = torch.tensor(batch_dec_target, dtype = torch.long, device='cuda')

        # Forward pass through encoder
        encoder_outputs, (encoder_hidden, encoder_cell) = encoder(batch_enc_input)
                 
        eh = torch.cat((encoder_hidden[0], encoder_hidden[1]), dim=1).unsqueeze(0)
        ec = torch.cat((encoder_cell[0], encoder_cell[1]), dim=1).unsqueeze(0)
        
        # Create initial decoder input (start with <sos> tokens for each sentence)
        dec_input = torch.tensor([word2idx['<sos>']] * batch_size, dtype=torch.long, device=device)
        dec_input = dec_input.unsqueeze(1)    # dec_input shape : (batch_size, 1)

        # Set initial decoder hidden state to the encoder's final hidden state
        decoder_hidden = encoder_hidden[:decoder.n_layers]

        # Determine if we are using teacher forcing this iteration
        # random.random() -> 0~1 사이의 난수 생성
        use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
        
        if use_teacher_forcing:
            for t in range(1, batch_dec_target.size(1)):
                
                decoder_output, eh, ec = decoder(dec_input, eh, ec, encoder_outputs)
                
                # Calculate and accumulate loss
                loss += criterion(decoder_output, batch_dec_target[:, t])

                # Teacher forcing: next input is current target
                # batch_dec_target의 전체 row의 t번째 값(정답)을 dec_input에 준다
                # dec_input shape : (batch_size, 1)
                dec_input = batch_dec_target[:, t].unsqueeze(1)                
        else:
            for t in range(1, batch_dec_target.size(1)):
                decoder_output, eh, ec = decoder(dec_input, eh, ec, encoder_outputs)
            
                # No teacher forcing: next input is decoder's own current output
                _, topi = decoder_output.topk(1)
            
                dec_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
                dec_input = dec_input.to(device)
            
                # Calculate and accumulate loss
                loss += criterion(decoder_output, batch_dec_target[:, t])
            
        # item() method extracts the loss’s value as a Python float     
        batch_loss = (loss.item() / int(batch_dec_target.size(1)))
        batch_ppl = np.exp(batch_loss)
        
        total_loss += batch_loss
        total_ppl += batch_ppl
        print_loss += loss.item()
        
        # Perform backpropagation
        loss.backward()
            
        # Adjust/Update model weights
        encoder_optimizer.step()
        decoder_optimizer.step()
        
        if (batch_idx + 1) % LOG_INTERVAL == 0:
            print("[epoch {} | step {}/{}] loss: {:.4f} (Avg. {:4f}) PPL: {:.4f} (Avg. {:.4f})".format(epoch+1,
                                                                                                       batch_idx+1,
                                                                                                       num_batches_per_epoch,
                                                                                                       batch_loss, total_loss/(batch_idx + 1),
                                                                                                       batch_ppl, total_ppl/(batch_idx + 1)))

    # check validation
    val_batches = batch_iter(test, batch_size)   

    encoder.cuda().eval()
    decoder.cuda().eval()
    val_loss = evaluate(encoder, decoder, val_batches, device, word2idx)
    print("[epoch {}] loss: {:.4f}".format(epoch+1, val_loss))    
    
    
#모델 저장
torch.save(encoder, dirpath + 'encoder_' + str(epoch + 1) + '.pt')
torch.save(decoder, dirpath + 'decoder_' + str(epoch + 1) + '.pt')
#total_loss_per_epoch = (total_loss / (batch_idx + 1))

[num_batches_per_epoch] 332


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  after removing the cwd from sys.path.


[epoch 1 | step 100/332] loss: 1.5400 (Avg. 2.097175) PPL: 4.6645 (Avg. 70.6520)
[epoch 1 | step 200/332] loss: 1.4916 (Avg. 1.789732) PPL: 4.4444 (Avg. 37.5489)
[epoch 1 | step 300/332] loss: 1.2401 (Avg. 1.641250) PPL: 3.4558 (Avg. 26.3207)
[epoch 1] loss: 0.6323
[epoch 2 | step 100/332] loss: 1.1720 (Avg. 1.234736) PPL: 3.2284 (Avg. 3.4578)
[epoch 2 | step 200/332] loss: 1.1783 (Avg. 1.187921) PPL: 3.2490 (Avg. 3.3034)
[epoch 2 | step 300/332] loss: 0.9995 (Avg. 1.152238) PPL: 2.7168 (Avg. 3.1894)
[epoch 2] loss: 0.5699
[epoch 3 | step 100/332] loss: 0.9477 (Avg. 1.012397) PPL: 2.5797 (Avg. 2.7644)
[epoch 3 | step 200/332] loss: 0.9286 (Avg. 0.972499) PPL: 2.5309 (Avg. 2.6584)
[epoch 3 | step 300/332] loss: 0.8084 (Avg. 0.939317) PPL: 2.2443 (Avg. 2.5727)
[epoch 3] loss: 0.5435
[epoch 4 | step 100/332] loss: 0.7353 (Avg. 0.805989) PPL: 2.0862 (Avg. 2.2462)
[epoch 4 | step 200/332] loss: 0.7085 (Avg. 0.769010) PPL: 2.0309 (Avg. 2.1659)
[epoch 4 | step 300/332] loss: 0.6138 (Avg. 0.73

##Step 7: Test the chatbot model
- inference: 질문을 임베딩 벡터로 변환해 모델에 입력하고 그에 따른 출력값을 문장으로 변환
- unfiltering: 특수문자를 단어들로부터 제거하여 원래 단어 스트링들만 추출

In [ ]:
def inference(question):
  #질문을 임베딩 벡터로 변환해 모델에 입력하고 그에 따른 출력값을 문장으로 변환

  #질문을 idx 들로 변환한다.
  sentence = tokenizer.tokenize(question)
  sentence.append('<eos>')
  sentence = sentence + (MAX_LEN+1 - len(sentence)) * ['<pad>']
  sentence_idx = [[word2idx[word] if word in word2idx else UNK_token for word in sentence]] #1 for unk_token
  
  #입력 문장의 idx 를 벡터로 변환하여 인코더의 input 으로
  enc_input = torch.tensor(sentence_idx , device=device)

  encoder_outputs, (encoder_hidden, encoder_cell) = encoder(enc_input)
  eh = torch.cat((encoder_hidden[0], encoder_hidden[1]), dim=1)
  ec = torch.cat((encoder_cell[0], encoder_cell[1]), dim=1)
  decoder_input = torch.tensor([word2idx['<sos>']], dtype=torch.long, device=device)

  #인코더의 input 과 <sos> 토큰을 decoder에 입력하여 다음 단어 예측 
  decoded_words = []
  for di in range(MAX_LEN):
    decoder_output, eh, ec = decoder(decoder_input, eh, ec, encoder_outputs)

    #가장 확률이 높은 단어 하나를 예측하고, 이전 입력에 연결하여 다음 예측에 사용
    topv, topi = decoder_output.data.topk(1)
    if topi.item() == EOS_token or topi.item() == PAD_token:
        decoded_words.append('<eos>')
        break
    else:
        decoded_words.append(idx2word[topi.item()])
    decoder_input = torch.tensor([topi], device=device)
    
  print('text:', decoded_words)
  return unfiltering(decoded_words)
  print(decoded_words)


In [ ]:
# 특수문자를 단어들로부터 제거
def unfiltering(text):
  origin_text = ''
  for words in text[:-2]:
    origin_text += words + ' '

  return origin_text.replace(' ##' ,'')


In [ ]:
print("챗봇과 대화")
while (1):
  user_input = input("문장을 입력하세요: ")
  user_input = str(user_input)

  if user_input == '/q':
    print("Quitting chat..")
    break;
  else:
    print("Bot: " + str(inference(user_input)))

챗봇과 대화
문장을 입력하세요: 오늘 비가 내려
text: ['당신', '##의', '삶', '##을', '응원', '##해', '드릴', '수', '있', '##어요', '##라고', '감히', '말', '##해', '봅니다', '.', '<eos>']
Bot: 당신의 삶을 응원해 드릴 수 있어요라고 감히 말해 봅니다 
문장을 입력하세요: 승주 결혼 축하해
text: ['무엇', '##을', '해줘', '##서', '잘', '##하', '##는', '것', '##이', '아니', '##에', '##요', '.', '<eos>']
Bot: 무엇을 해줘서 잘하는 것이 아니에요 
문장을 입력하세요: 우리 모두 화이팅하자
text: ['좋', '##은', '느낌', '##들', '##의', '사랑', '##이', '##네', '##요', '.', '<eos>']
Bot: 좋은 느낌들의 사랑이네요 
문장을 입력하세요: 예쁜 사랑하세요
text: ['진정', '##으로', '사랑', '##한다', '##면', '그럴', '##수도', '있', '##을', '##거', '같', '##아', '##요', '.', '<eos>']
Bot: 진정으로 사랑한다면 그럴수도 있을거 같아요 
문장을 입력하세요: 오늘은 기분이 울적해요
text: ['거리', '##를', '걸어', '##보', '##세요', '.', '<eos>']
Bot: 거리를 걸어보세요 
문장을 입력하세요: /q
Quitting chat..
